# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv file as DataFrame
weather_data = pd.read_csv('../output_data/cities.csv')
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kismayo,-0.3582,42.5454,82.36,78,99,13.78,SO,1650146868
1,Lompoc,34.6391,-120.4579,68.14,73,100,12.66,US,1650146868
2,New Norfolk,-42.7826,147.0587,58.91,75,100,2.42,AU,1650146868
3,Äänekoski,62.6946,25.8180,30.67,65,25,3.58,FI,1650146868
4,Busselton,-33.6500,115.3333,55.62,76,16,9.19,AU,1650146868


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Create Humidity Heatmap

# Use Lat and Lng as locations and Humidity as weight
locations = weather_data[['Lat', 'Lng']] 
humidity = weather_data['Humidity'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer (max_intensity at the maximum humidity level of 100
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=3)
 
# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#* Narrow down the DataFrame to find your ideal weather condition. For example:

# * A max temperature lower than 80 degrees but higher than 70.

# * Wind speed less than 10 mph.

# * Zero cloudiness.

# * Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

ideal_weather_data = weather_data[
    (weather_data['Max Temp']<80) & 
    (weather_data['Max Temp'] >70) &
    (weather_data['Wind Speed'] < 10) &
    (weather_data['Cloudiness'] == 0)]


# Resetting index of new DataFrame
ideal_weather_data.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jhābua,22.7667,74.6000,79.36,39,0,7.09,IN,1650146875
1,Antalaha,-14.9003,50.2788,73.18,84,0,7.07,MG,1650146890
2,Vao,-22.6667,167.4833,76.60,70,0,8.03,NC,1650146890
3,Viramgam,23.1225,72.0484,79.38,58,0,7.52,IN,1650146914
4,Ratnagiri,16.9833,73.3000,79.07,72,0,1.54,IN,1650146931


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named `hotel_df`
hotel_df = ideal_weather_data.copy()

# Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ''
hotel_df = hotel_df.reset_index(drop=True)

In [6]:
# Set parameters to search for hotels with 5000 meters
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params.update({'location': f'{row["Lat"]}, {row["Lng"]}'})
    response = requests.get(base_url, params=params).json()
    results = response.get('results',[])
    if results:
        hotel_name = results[0]['name'] 
    else:
        hotel_name = 'Hotel not Found'
    hotel_df['Hotel Name'][index] = hotel_name
        
hotel_df.reset_index(drop=True)

/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Jhābua,22.7667,74.6000,79.36,39,0,7.09,IN,1650146875,aditi residency
1,Antalaha,-14.9003,50.2788,73.18,84,0,7.07,MG,1650146890,Hotel Vitasoa
2,Vao,-22.6667,167.4833,76.60,70,0,8.03,NC,1650146890,Hotel Kou-Bugny
3,Viramgam,23.1225,72.0484,79.38,58,0,7.52,IN,1650146914,'Hotel Asopalav & Guest House-honest
4,Ratnagiri,16.9833,73.3000,79.07,72,0,1.54,IN,1650146931,Hotel Landmark


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Finding out if the hotels and their cities are in the Northern or Southern Hemisphere

northern_hotels = hotel_df.loc[hotel_df['Lat'] >= 0]
northern_hotels

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Jhābua,22.7667,74.6000,79.36,39,0,7.09,IN,1650146875,aditi residency
3,Viramgam,23.1225,72.0484,79.38,58,0,7.52,IN,1650146914,'Hotel Asopalav & Guest House-honest
4,Ratnagiri,16.9833,73.3000,79.07,72,0,1.54,IN,1650146931,Hotel Landmark


In [10]:
# Finding out if the hotels and their cities are in the Northern or Southern Hemisphere

southern_hotels = hotel_df.loc[hotel_df['Lat'] < 0]
southern_hotels

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Antalaha,-14.9003,50.2788,73.18,84,0,7.07,MG,1650146890,Hotel Vitasoa
2,Vao,-22.6667,167.4833,76.60,70,0,8.03,NC,1650146890,Hotel Kou-Bugny


In [11]:
# Setting the ideal humidity level at 50% 

ideal_humidity = weather_data.loc[weather_data['Humidity']<50]
ideal_humidity.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Wagar,16.1525,36.2032,85.17,40,70,7.31,SD,1650146869
1,Makkah al Mukarramah,21.4267,39.8261,81.34,36,0,1.23,SA,1650146870
2,Lar,26.2039,83.9717,83.57,12,20,5.41,IN,1650146871
3,Catalina,32.5056,-110.9211,88.92,9,50,5.01,US,1650146871
4,Ketchikan,55.3422,-131.6461,44.58,38,26,4.05,US,1650146872
...,...,...,...,...,...,...,...,...,...
85,Jalu,29.0331,21.5482,75.97,22,76,16.78,LY,1650146929
86,Khasan,42.4308,130.6434,45.64,47,99,3.42,RU,1650146930
87,Tucumcari,35.1717,-103.7250,63.95,23,0,8.05,US,1650146930
88,Mexicali,32.6519,-115.4683,91.26,11,0,21.30,MX,1650146930


# VacationPy Analysis

- The dataframes and maps of cities around the world which was generated using uniform sampling show that the hotels and ideal weather cities are quite evenly divided into the Northern (3) and Southern (2) hemispheres.

- India has the most (3) cities in the ideal weather dataframe out of the 5 in total.

- In the ideal weather dataframe only 1 city has humitiy levels below 50%: Jhābua in India. Looking at this and the humidity  heatmap there seems to be high levels of humidity around the world. If we set ideal humidity levels at 50%, the original dataframe of 555 is reduced to 90 cities with humitiy levels below 50%. 